In [37]:
# libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib as mpl
import pandas as pd
import sys
import calendar
from matplotlib.lines import Line2D

'''
src_path = sys.path[0].replace("figures\visualization\script", "src")
#replace notebook as scripts
data_path = sys.path[0].replace("figures\visualization\script", "data")
if src_path not in sys.path:
    sys.path.append(src_path)

out_path = sys.path[0].replace("figures\visualization\script", "output")
'''

# set default params of plot
rc_={
    "figure.dpi":300,
    "font.size":10,
    "axes.facecolor":"white",
    "savefig.facecolor":"white",
    "text.usetex":True,
}

color_dic_glb={
        'MPC-GT':'seagreen',# aquamarine
        'MPC-Prediction':'navy',
        'MPC-Heuristic':'steelblue',
        'MPC-Naive':'slategray',
        'MSC-GT':'orangered', #green
        'MPC-Disturbance':'purple',
        'MSC-Naive':'grey'
    }
color_dic_glb_w={
        'MPC-GT':'w',# aquamarine
        'MPC-Prediction':'w',
        'MPC-Heuristic':'w',
        'MPC-Naive':'w',
        'MSC-GT':'w', #green
        'MPC-Disturbance':'w',
        'MSC-Naive':'w'
    }
marker_dic_glb_w={
        'MPC-GT':'_',
        'MPC-Prediction':'^',
        'MPC-Heuristic':'x',
        'MPC-Naive':'v',
        'MPC-Disturbance':'+',
        'MSC-GT':'_',
        'MSC-Naive':'_'
    }

marker_dic_glb={
        'MPC-GT':'_',
        'MPC-Prediction':'_',
        'MPC-Heuristic':'_',
        'MPC-Naive':'_',
        'MPC-Disturbance':'_',
        'MSC-GT':'_',
        'MSC-Naive':'_'
    }

In [38]:
# process the original data
def pre_process(df,key,duration_key):
    df.set_index('id')
    
    df=df[['strategy','pred_model',duration_key,key]]
    df=df.replace('optimal',"MPC")
    df=df.replace("Simple","Heuristic")
    df['label']=df['strategy']+'-'+df['pred_model']
    data=dict()
    
    for i in df[duration_key].unique():
        try:
            for label in df['label'].unique():
                data[label]=df[((df[duration_key]==i)) & (df['label']==label)][key].values[0]
            
            upper=df[((df[duration_key]==i)) & (df['label']=='MSC-GT')][key].values[0]
            lower=df[((df[duration_key]==i)) & (df['label']=='MPC-GT')][key].values[0]
            '''
            #assert upper>lower
            prediction=df[((df['week_of_year']==i)) & (df['label']=='MPC-Prediction')][key].values[0]
            heuristic=df[((df['week_of_year']==i)) & (df['label']=='MPC-Heuristic')][key].values[0]
            naive=df[((df['week_of_year']==i)) & (df['label']=='MPC-Naive')][key].values[0]
            '''
        except:
            pass
        
        upper_id=df[((df[duration_key]==i)) & (df['label']=='MSC-GT')][key].index
        lower_id=df[((df[duration_key]==i)) & (df['label']=='MPC-GT')][key].index
        
        for label in df["label"].unique():
            data[label+'_id']=df[((df[duration_key]==i)) & (df['label']==label)][key].index
        '''
        prediction_id=df[((df['week_of_year']==i)) & (df['label']=='MPC-Prediction')][key].index
        heuristic_id=df[((df['week_of_year']==i)) & (df['label']=='MPC-Heuristic')][key].index
        naive_id=df[((df['week_of_year']==i)) & (df['label']=='MPC-Naive')][key].index 
        '''
        
        new_key='relative_'+key
        df.loc[lower_id,new_key]=0
        df.loc[upper_id,new_key]=100
        
        for label in df["label"].unique():
            df.loc[data[label+'_id'],new_key]=(data[label]-lower)/(upper-lower)*100
        '''    
        df.loc[prediction_id,new_key]=(prediction-lower)/(upper-lower)*100
        df.loc[heuristic_id,new_key]=(heuristic-lower)/(upper-lower)*100
        df.loc[naive_id,new_key]=(naive-lower)/(upper-lower)*100
        '''
    #df_new=df.drop(df[df['label']=='MPC-Naive'].index)
    print(df)
    df=df.reindex()
    invalid_duration_list=df[(df[new_key]>100)|(df[new_key]<0)|(df[key].isna())].index
    invalid_duration_list=df.iloc[invalid_duration_list][duration_key].unique()
    df['is_valid']=df[duration_key].apply(lambda x: True if x not in invalid_duration_list else False)
    return df

In [39]:
fn=r'L:\Coding_project\Energy_grid_new\output\experiments\thesis_value_of_information\6h-bat-Oct-disturbance-type.xlsx'

In [47]:
df=pd.read_excel(fn,sheet_name="Sheet1")
df["relative_OPEX"]=None
mpc_gt=df[(df.strategy=='optimal')&(df.pred_model=='GT')]['OPEX'].values[0]
msc_gt=df[(df.strategy=='MSC')&(df.pred_model=='GT')]['OPEX'].values[0]
diff=msc_gt-mpc_gt
df["relative_OPEX"]=(df['OPEX']-mpc_gt)/diff*100
df["disturbance_scale"]=df["disturbance_scale"]*100

In [41]:
df_grouped=df.groupby(by=["p_grid_max_method"])

In [42]:
df_grouped.groups.keys()

dict_keys(['by_execution', 'minimize_cap', 'zero'])

In [54]:
df_dic={}
df_sub_dic={
    "n":None,
    "n+":None,
    "n-":None
}

In [50]:
key="minimize_cap"
df_dic[key]=df_grouped.get_group(key)

In [52]:
df_sub_grouped=df_dic[key].groupby(by=["disturbance_rule"])

In [55]:
df_sub_dic["n"]=df_sub_grouped.get_group("uniform")
df_sub_dic["n+"]=df_sub_grouped.get_group("uniform_pos")
df_sub_dic["n-"]=df_sub_grouped.get_group("uniform_neg")

In [ ]:
loc_dic={
    "n":-0.001,
    "n+":0,
    "n-":0.001
}
color_dic={
    "n":'darkred',
    "n+":'orangered',
    "n-":'green'
}
marker_dic={
    "n":'_',
    "n+":'_',
    "n-":'_'
}
figsize=(10,5)
fig,ax=plt.subplots(figsize=figsize)
for key in loc_dic.keys:
    scatter_x=np.array(df_sub_dic[key]["disturbance_scale"])
    scatter_y=np.array(df_sub_dic[key]["relative_OPEX"])
    ax.scatter(scatter_x, scatter_y, label=key,c=color_dic[key],\
            marker=marker_dic[key],s=10)
